In [1]:
! pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 75.6 MB/s eta 0:00:00


In [2]:
## pdf-Datei anschauen: Die ersten 7 Seiten enthalten Deckblatt + Inhaltsverzeichnis - das kann also übersprungen werden
## Letzten 3 Seiten sind auch unnötig

# Als erstes die Formatinformation extrahieren: Also Text mit Schriftgröße und Dicke der Schrift, um später die Überschriften zu finden und
# zur Gliederung verwenden zu können

# Text ab siebter Seite extrahieren inkl. Formatinfos:

import fitz

def extract_text_with_formatting(pdf_path, skip_pages=7):
    doc = fitz.open(pdf_path)
    text_data = []

    for page_num in range(skip_pages, 158):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]  # Extrahiert Textblöcke

        for block in blocks:
            if block["type"] == 0:  # Textblock
                for line in block["lines"]:
                    for span in line["spans"]:
                        # Speichere Text + Schriftgröße + Fettdruck-Info
                        text_data.append({
                            "text": span["text"],
                            "size": span["size"],  # Schriftgröße
                            "bold": span["flags"] & 2 != 0  # Prüft, ob der Text fett ist
                        })

    return text_data

# PDF-Dateipfad
pdf_path = "/content/Wahlprogramm_Die_Gruenen.pdf"

# Text mit Formatierungsinformationen extrahieren
text_data = extract_text_with_formatting(pdf_path)

# Beispiel: Zeige die ersten paar Textabschnitte mit Formatierung
for item in text_data[:1000]:
    print(f"Text: {item['text']}, Size: {item['size']}, Bold: {item['bold']}")


Text: BÜNDNIS 90 / DIE GRÜNEN, Size: 6.0, Bold: False
Text: B, Size: 46.14289855957031, Bold: False
Text: ei den Bundestagswahlen am 23. Februar ste-, Size: 9.0, Bold: False
Text: hen Sie, liebe Bürgerinnen und Bürger, vor einer , Size: 9.0, Bold: False
Text: wichtigen Entscheidung. Es geht darum, wer für , Size: 9.0, Bold: False
Text: unser Land in dieser herausfordernden Zeit Regierungsver-, Size: 9.0, Bold: False
Text: antwortung trägt., Size: 9.0, Bold: False
Text: Jetzt kommt es auf die Kraft unseres Zusammenlebens an, , Size: 9.0, Bold: False
Text: in dem die Menschen auch in diesen herausfordernden Zei-, Size: 9.0, Bold: False
Text: ten so viel einbringen und leisten. Wir nehmen diese Kraft , Size: 9.0, Bold: False
Text: als Auftrag an die Politik, unser Land für alle Menschen in , Size: 9.0, Bold: False
Text: den kommenden Jahren besser, gerechter und einfacher zu , Size: 9.0, Bold: False
Text: machen. Als Auftrag, dass es hier einfach funktioniert und , Size: 9.0, Bold: False


In [3]:
## Output anschauen: Normale Schrift ist in Schriftgröße 9, Überschriften sind größer, Seitenzahlen sind kleiner oder größer
## Kopfzeile in Schriftgröße 6
## Präambel und Kapitel X immer beschriftet
## Recht viele Bindestriche leider
## Überschriften durch große Buchstaben gekennzeichnet, die zum Text gehören


import pandas as pd
import re


def split_sections(text_data, size_target=9, min_size=9, bold_threshold=True):
    sections = []
    current_section = []

    for item in text_data:
        text = item["text"]
        text = text.replace("-", "") # Bindestriche entfernen

        if re.match(r"^(Präambel|Kapitel \d+)$", text):  # Überspringe "Präambel" und "Kapitel X"
            continue

        if len(text) == 1 and item["size"] > min_size and item["size"] < 100: # Wenn Text nur einzelner Buchstabe -> hinzufügen
                current_section.append(text)
                continue

        if item["size"] > min_size and item["size"] < 110 or item["bold"] == bold_threshold: # Überschriften überspringen + Abschnitt definieren
            if current_section:
                sections.append("".join(current_section).strip())
            current_section = []  # Abschnitt zurücksetzen

        elif item["size"] == size_target: # Text speichern in Abschnitt
            current_section.append(text)

        elif text.isdigit() and len(text) <= 4:  # Seitenzahlen überspringen
            continue

    if current_section:
        final_section = "".join(current_section).strip() # Abschnitt hinzufügen in Liste
        sections.append(final_section)

    return sections


# Abschnitte extrahieren
sections = split_sections(text_data)

# Erstelle Tabelle mit Abschnitten
df = pd.DataFrame(sections, columns=["Abschnitt"])

print(df[:10])

                                           Abschnitt
0  Bei den Bundestagswahlen am 23. Februar stehen...
1  Es geht jetzt darum, diese Kraft als Zukunftsk...
2  Die Menschen in Deutschland sind zu Recht stol...
3  Unternehmen brauchen gute Wettbewerbsbedingung...
4  Der Mangel an Arbeits und Fachkräften ist eine...
5  Der europäische Binnenmarkt mit seinen 450 Mil...
6  Funktionierende Finanzmärkte sind ein essenzie...
7  Deutschland und Europa müssen bei den Innovati...
8  Unser Anspruch ist es, dass Deutschland ein st...
9  Die ökonomische Kraft unseres Landes liegt in ...


In [5]:
# Speichern der Tabelle als CSV-Datei
df.to_csv("/content/gruene.csv", index=False, encoding="utf-8")